The estimation is done in two parts: a structural/macro model and models/techniques for adjusting and re-weighting polls. The structural model uses economic indicators and incumbency (previous year winner) as independent variables; the output is a set of vote share estimates per party per province that are then used as priors for the province-level simulations. The re-weighting of the polling data includes adjustments to weight polls from recent periods higher as well as those with larger sample sizes. 

I also attempted a model for identifying housing effects and adjusting polls accordingly, however this model didn't prove successful and needs more time to develop so I exclude it from the estimation. 

The polls used in the estimation are the province and region-level ones. When testing with the national level polls in my current setup, they caused a significant drift for some provinces (e.g. where SSP has significant vote share); to include national polls, some more preprocessing (i.e. estimating a trend line) would be needed at the very least. I exclude national polls for now for this reason.   

The results of this notebook are stored in `/model_outputs`.

In [1]:
import pandas as pd
import os
import numpy as np
import pymc3 as pm
import json


In [2]:
folder = '/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/raw_data'

filelist = [file for file in os.listdir(folder) if file.startswith('dataland')]
for file in filelist:
    exec("%s = pd.read_csv('%s')" % (file.split('.')[0], os.path.join(folder,file)))

Functions.

In [3]:
def create_structural_data(df_y, df_X_econ):
    """Steps:
    1. Process data
    2. Join processed X data to y data
    """
    # create year column
    df_X_econ['year'] = pd.DatetimeIndex(df_X_econ['date']).year

    # create last year province winner; captures "incumbency" effect for the province 
    df_y['previous_year_provincial_winner'] = df_y.groupby(['province'])['provincial_winner'].shift()
    df_y['previous_year_provincial_winner'] = df_y['previous_year_provincial_winner'].fillna('None')
    df_y = pd.get_dummies(df_y, columns=['previous_year_provincial_winner'], drop_first=True)
    df_y[['previous_year_provincial_winner_cc',
       'previous_year_provincial_winner_dgm',
       'previous_year_provincial_winner_pdal',
       'previous_year_provincial_winner_ssp']] = df_y[['previous_year_provincial_winner_cc',
       'previous_year_provincial_winner_dgm',
       'previous_year_provincial_winner_pdal',
       'previous_year_provincial_winner_ssp']].astype(int) 

    # join all data together
    return df_y.merge(df_X_econ, on='year', how='inner')

def process_poll_data(df_polls, df_electoral):

    df_electoral.rename(columns={'election_cycle':'year'}, inplace=True)
    df_merged = df_polls.merge(df_electoral[['year', 'election_day']], on=['year'], how='left')

    # convert object to datetime
    df_merged['date_conducted'] = pd.to_datetime(df_merged['date_conducted'])
    df_merged['date_published'] = pd.to_datetime(df_merged['date_published'])

    # create days to election col 
    df_merged['election_day'] = pd.to_datetime(df_merged['election_day'])
    df_merged['days_to_election'] = (df_merged['election_day'] - df_merged['date_published']).dt.days

    # normalise vars
    # df_merged['days_to_election'] = (df_merged['days_to_election'] - df_merged['days_to_election'].min()) / (df_merged['days_to_election'].max() - df_merged['days_to_election'].min())
    # df_merged['sample_size'] = (df_merged['sample_size'] - df_merged['sample_size'].mean()) / df_merged['sample_size'].std()

    # drop any polls where there are 0 vote shares within geos, but non-zero average (needed for modelling step to not have 0 shares)
    # Note: indicates the poll left a running candidate out
    party_list = ['cc_poll_share', 'dgm_poll_share', 'pdal_poll_share']# exclude ssp as this party doesn't run in all geos
    geo_means = df_merged[['geography', 'year']+party_list].groupby(['geography', 'year']).agg({'cc_poll_share':'mean', 'dgm_poll_share':'mean', 'pdal_poll_share':'mean'}).reset_index()
    geo_means[party_list] = geo_means[party_list].add_suffix("_geo_mean")

    geo_means = geo_means.rename(columns={c: c+'_geoMean' for c in geo_means.columns if c not in ['geography', 'year']})

    df_merged_w_means = df_merged.merge(geo_means, on=['geography', 'year'], how='left')
    for p in party_list:
        df_merged_w_means = df_merged_w_means[(df_merged_w_means[p] != 0) & (df_merged_w_means[p+'_geoMean'] != 0)]
    
    df_final = df_merged_w_means.drop(list(df_merged_w_means.filter(regex = 'geoMean')), axis = 1)

    return df_final.drop(['election_day'], axis=1)

In [4]:
def run_structural_model(df, indep_cols, dep_cols):

    df_indep = df[indep_cols]
    df_dep =  df[dep_cols] 

    election_idx, election = df.year.factorize(sort=True)

    # set up coords
    coords = {
        "dep": df_dep.columns,
        "sample": np.arange(len(df_dep)),
        "indep": df_indep.columns,
        "election": election,
    }

    # set up model
    with pm.Model(coords=coords) as structural_model:
        # data inputs
        ivs = pm.Data('ivs', df_indep.values, dims=('sample', 'indep')) # NxK
        dv = pm.Data('dv', df_dep.values, dims=('sample', 'dep')) # NxP

        # naive prior for target distributions
        theta = pm.Dirichlet('theta', np.ones(len(dep_cols)))
        # scaling factor to adjust probabilities
        exptheta = pm.Deterministic('exptheta', pm.math.exp(theta))

        # varying b0 intercept by election year
        b0 = pm.Normal('b0', 0, 1, dims=('election', 'dep')) # NxP

        # betas
        b = pm.Gamma('b', 1, 0.5, dims=('indep', 'dep')) # KxP

        # linear combination of effects; apply logit link function 
        a = pm.Deterministic('a', pm.math.invlogit(b0[election_idx]+pm.math.dot(ivs, b))*exptheta) # results in NxP

        # Dirichlet likelihood to model the observed dependent variables
        lik = pm.Dirichlet('likelihood', a=a, observed=dv, dims=('sample', 'dep'))

        # perform inference using the data  
        trace = pm.sample(draws=1000, tune=100, chains=2, random_seed=42)  

        # sampling data to get posterior distributions
        post_pred = pm.sample_posterior_predictive(trace) 
    
    return structural_model, trace, post_pred

def run_house_effects_model(df, poll_entries_list, prior_alpha_list):

    # set up coords
    indep_polls = ['days_to_election', 'mode', 'sponsor', 'sample_size']
    dep_polls = poll_entries_list

    # one hot encode
    df_indep_polls = pd.get_dummies(df[indep_polls], columns=['mode', 'sponsor']).astype(int).drop(columns=['mode_Online']) # drop one; Note: don't need to drop for sponsor as all 0s indicate NA=no sponsor
    df_dep_polls =  df[dep_polls] 

    pollster_idx, pollster = df.pollster.factorize(sort=True)

    coords_polls = {
        "dep": df_dep_polls.columns,
        "sample": np.arange(len(df_dep_polls)),
        "indep": df_indep_polls.columns,
        "pollster": pollster,
    }

    with pm.Model(coords=coords_polls) as poll_model:
        # data inputs
        ivs = pm.Data('ivs', df_indep_polls.values, dims=('sample', 'indep')) # NxK
        dv = pm.Data('dv', df_dep_polls.values, dims=('sample', 'dep')) # NxP

        # use posterior pred distribution from structural model as prior
        theta = pm.Dirichlet('theta', a=prior_alpha_list)

        # varying intercept by pollster, i.e. pollster effect
        b0 = pm.Normal('b0', 0, 1, dims=('pollster', 'dep')) # NxP

        b = pm.Normal('b', 0, 1, dims=('indep', 'dep')) # KxP

        a = pm.Deterministic('a', theta * pm.math.invlogit(b0[pollster_idx]+pm.math.dot(ivs, b))) # results in NxP

        # Dirichlet likelihood to model the observed dependent variables
        lik = pm.Dirichlet('likelihood', a=a, observed=dv, dims=('sample', 'dep'))

        # perform inference using the data  
        trace = pm.sample(draws=1000, tune=100, chains=2, random_seed=42)  

        # sampling data from before and after data is available   
        post_pred = pm.sample_posterior_predictive(trace) 

    return poll_model, trace, post_pred, pollster

In [5]:
def calculate_avg_vote_shares(post_pred):
    return np.mean(np.mean(post_pred['likelihood'], axis=0), axis=0)

def calculate_avg_house_effect(trace_poll):
    return np.mean(np.mean(trace_poll['b0'], axis=0), axis=0)

In [6]:
def get_house_effects(b0_samples, pollster_names, poll_entries_list):
    mean_pollster_effects = np.mean(b0_samples, axis=0)
    pollster_effects_df = pd.DataFrame(data=mean_pollster_effects, columns=poll_entries_list)
    pollster_effects_df['pollster'] = pollster_names
    pollster_effects_df.columns = [col + '_adjustment' if col != 'pollster' else col for col in pollster_effects_df.columns]
    return pollster_effects_df

def adjust_polls_with_house_effect(df_polls_input, df_pollster_effects, poll_entries_list):
    df_adjusted = df_polls_input.merge(df_pollster_effects, on='pollster', how='left')

    for prefix in poll_entries_list:
        columns_to_multiply = [col for col in df_adjusted.columns if col.startswith(prefix)]
        df_adjusted[prefix] = df_adjusted[columns_to_multiply].product(axis=1)
        df_adjusted = df_adjusted.drop(columns_to_multiply[1], axis=1)

    df_adjusted[poll_entries_list] = df_adjusted[poll_entries_list].div(df_adjusted[poll_entries_list].sum(axis=1), axis=0)

    return df_adjusted

In [7]:
def redistribute_polls(df_input, col_to_redistribute, party_col_list):
    """For now, assume equal split of undecided share to participating parties, 
    i.e. no effect of undecided on election outcome.
    For provinces where no SSP party is running, equally redistribute Nat'l polls
    to participating parties.
    """
    df = df_input.copy()

    # Redistribute values row by row
    for index, row in df.iterrows():
        sum_to_redistribute = row[col_to_redistribute]
        num_remaining_parties = len(party_col_list)
        redistributed_value = sum_to_redistribute / num_remaining_parties
        
        # Redistribute the values equally to the remaining party columns for this row
        for party in party_col_list:
            df.at[index, party] += redistributed_value
    
    return df.drop(columns=[col_to_redistribute], axis=1)
    

def get_ewma_weights(df_input, group_column, party_col_list, alpha = 0.2):
    """Give newer polls more weight."""

    df = df_input.copy()
    for col in party_col_list:
        df[col] = df.groupby(group_column)[col].transform(lambda x: x.ewm(alpha=alpha, adjust=False).mean())
    
    return df[[group_column]+party_col_list]
        
def get_sample_size_weights(df_input, group_column, target_column, party_col_list):
    """Give larger sample size polls more weight."""

    df = df_input.copy()
    # Calculate the square root of each individual row in the target column
    df['sqrt_' + target_column] = np.sqrt(df[target_column])
    
    # Group by the specified column, calculate the median of the target column within each group, and reset the index
    median_values = df.groupby(group_column)[target_column].transform('median')
    
    # Divide sqrt_target_column by sqrt_median_value to get the result
    result = df['sqrt_' + target_column] / np.sqrt(median_values)

    df['wi_' + target_column] = result
    df['w_sum_' + target_column] = df.groupby(group_column)['wi_' + target_column].transform('sum')
    df['sample_size_weight'] = df['wi_' + target_column] /  df['w_sum_' + target_column]

    # Multiply weights with each party poll value 
    for col in party_col_list:
        df[col] = df[col] * df['sample_size_weight']
    
    return df[[group_column]+party_col_list]

In [72]:
def run_forecast_pipeline(df_structural, df_structural_scenario, df_polls, df_poll_scenario):
    """Given data on a province and a scenario level, forecast election day results.
    """

    ### Structural part
    # ivs
    indep_cols = [
            'year_on_year_gdp_pct_change', 
            'unemployment_rate', 
            'year_on_year_inflation', 
            'year_on_year_stock_mkt_pct_change',
            'previous_year_provincial_winner_cc',
            'previous_year_provincial_winner_dgm',
            'previous_year_provincial_winner_pdal',
            'previous_year_provincial_winner_ssp'
            ]

    # dvs; remove ssp if not running in the province
    if df_structural.ssp_share.mean() == 0:
        parties_list = ['cc_share', 'dgm_share', 'pdal_share'] 
    else:
        parties_list = ['cc_share', 'dgm_share', 'pdal_share', 'ssp_share']

    structural_model, trace, post_pred = run_structural_model(df=df_structural, dep_cols=parties_list, indep_cols=indep_cols)

    # Update structural model
    df_w_sc = pd.concat([df_structural, df_structural_scenario]).fillna(method='ffill')
    min_year = df_structural['year'].min()
    df_w_sc = df_w_sc[df_w_sc['year'] != min_year]

    # forecast structural posterior
    new_values = df_w_sc[indep_cols].values
    with structural_model:
        # Switch out the observed dataset
        pm.set_data({'ivs': new_values})
        trace_sc = pm.sample(draws=1000, tune=100, chains=2)
        post_pred_sc = pm.sample_posterior_predictive(trace_sc)
    
    prior_alpha_list = calculate_avg_vote_shares(post_pred_sc).tolist()
    # priors_list_df = pd.DataFrame(prior_alpha_list, columns=['alpha_prior'])


    ### Dynamic part
    # get the participating parties in province
    geo_list = df_polls.geography.unique().tolist()[1]
    # prov = [item for item in geo_list if item != 'National'][0]
    # remove ssp if not running in the province
    if df_polls[df_polls['geography']==geo_list].ssp_poll_share.mean() == 0:
        poll_entries_list = ['cc_poll_share', 'dgm_poll_share', 'pdal_poll_share']
    else:
        poll_entries_list = ['cc_poll_share', 'dgm_poll_share', 'pdal_poll_share', 'ssp_poll_share']

    # poll_model, trace_poll, post_pred_poll, pollster = run_house_effects_model(df_polls, poll_entries_list, prior_alpha_list)

    # append latest polls 
    df_polls_w_sc = pd.concat([df_polls, df_poll_scenario]).fillna(method='ffill').reset_index(drop=True)

    # # extract house effects and adjust polls
    # # TODO: takes too long and doesn't converge for all dataframes; would replace with frequentist model
    # b0_samples = trace_poll['b0']
    # df_pollster_effects = get_house_effects(b0_samples, pollster, poll_entries_list)
    # df_polls_pollster_adj = adjust_polls_with_house_effect(df_polls_w_sc, df_pollster_effects, poll_entries_list)

    # create avg poll shares per date in df
    df_polls_redistr = redistribute_polls(df_polls_w_sc, 'undecided_poll_share', poll_entries_list)#redistribute_polls(df_polls_pollster_adj, 'undecided_poll_share', poll_entries_list)
    if df_polls.ssp_poll_share.mean() == 0:
        df_polls_redistr = redistribute_polls(df_polls_redistr, 'ssp_poll_share', poll_entries_list)

    # df_polls_amp = df_polls_redistr[df_polls_redistr['geography'].isin(['National',prov])]
    df_polls_redistr = df_polls_redistr[['year', 'date_published', 'geography', 'sample_size']+poll_entries_list]

    num_cols = poll_entries_list+['sample_size']
    cat_cols = ['geography', 'year']

    # average polls from same geography and date
    df_polls_redistr = df_polls_redistr.groupby('date_published').agg({
        **{col: 'mean' for col in num_cols},
        **{col: 'first' for col in cat_cols}
    }).reset_index()

    # give newer polls more weight
    time_adj = df_polls_redistr.groupby(['year']).apply(lambda x: (
            (get_ewma_weights(x, 'geography', poll_entries_list)[poll_entries_list]) 
        )).reset_index()

    # give larger samples more weight
    ss_adj = df_polls_redistr.groupby(['year', 'date_published']).apply(lambda x: (
                (get_sample_size_weights(x, 'geography', 'sample_size', poll_entries_list).groupby(['geography'])[poll_entries_list].sum())
            )).reset_index()

    # combine weights 
    final_daily_averages = (time_adj[poll_entries_list] + ss_adj[poll_entries_list]) / 2
    final_daily_averages = pd.merge(ss_adj[['year', 'date_published']], final_daily_averages, left_index=True, right_index=True, how='inner')

    return prior_alpha_list, final_daily_averages




In [31]:
# load and process raw data
df_structural = create_structural_data(df_y=dataland_election_results_1984_2023, df_X_econ=dataland_economic_data_1984_2023)
df_polls = process_poll_data(dataland_polls_1984_2023_updated, dataland_electoral_calendar)

Run pipeline to collect model outputs.

In [85]:
scenario = 'A'
struct_dict_A = {} # stores province level results
poll_results_A = [] # stores province level results

for province in df_structural.province.unique():
    df_str_province = df_structural[df_structural['province'] == province]
    if province in ['Amperville', 'Circuiton', 'Electropolis', 'Voltagea']:
        region = 'Circuit Confederation'
    elif province in ['Cerebrica', 'Cortexia', 'Neuronia']:
        region = 'Synapse Territories'
    else: region = 'Metaflux Realm'

    df_polls_province = df_polls[df_polls['geography'].isin([region, province])]
    df_structural_scenario = dataland_economic_data_2024_scenarios[dataland_economic_data_2024_scenarios['scenario'] == scenario].drop('scenario', axis=1)
    df_structural_scenario['year'] = pd.DatetimeIndex(df_structural_scenario['date']).year
    df_poll_scenario = dataland_polls_2024_scenarios[
        (dataland_polls_2024_scenarios['scenario'] == scenario) & 
        (dataland_polls_2024_scenarios['geography'].isin([region, province]))
        ].drop('scenario', axis=1)
    # run model
    priors_list, final_daily_averages = run_forecast_pipeline(df_structural=df_str_province, 
                            df_structural_scenario = df_structural_scenario, 
                            df_polls = df_polls_province, 
                            df_poll_scenario = df_poll_scenario
                            )
    struct_dict_A[province] = priors_list
    final_daily_averages.insert(1, 'province', province)
    poll_results_A.append(final_daily_averages)

/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 9 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 9 seconds.
The acceptance probability does not match the target. It is 0.9006744419476235, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


...

/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 10 seconds.
The acceptance probability does not match the target. It is 0.8925907377790114, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 11 seconds.
The acceptance probability does not match the target. It is 0.9041758748635111, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 13 seconds.
The acceptance probability does not match the target. It is 0.9122415421849727, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9287460833921254, but should be close to 0.8. Try to increase the number of tuning steps.


In [93]:
# save model outcomes
with open('../model_outputs/struct_dict_A.json', 'w') as file:
    json.dump(struct_dict_A, file)

df_poll_results_A = pd.concat(poll_results_A)
df_poll_results_A.to_csv("../model_outputs/poll_results_A.csv", index=False)


In [98]:
scenario = 'B'
struct_dict_B = {} # stores province level results
poll_results_B = [] # stores province level results

for province in df_structural.province.unique():
    df_str_province = df_structural[df_structural['province'] == province]
    if province in ['Amperville', 'Circuiton', 'Electropolis', 'Voltagea']:
        region = 'Circuit Confederation'
    elif province in ['Cerebrica', 'Cortexia', 'Neuronia']:
        region = 'Synapse Territories'
    else: region = 'Metaflux Realm'

    df_polls_province = df_polls[df_polls['geography'].isin([region, province])]
    df_structural_scenario = dataland_economic_data_2024_scenarios[dataland_economic_data_2024_scenarios['scenario'] == scenario].drop('scenario', axis=1)
    df_structural_scenario['year'] = pd.DatetimeIndex(df_structural_scenario['date']).year
    df_poll_scenario = dataland_polls_2024_scenarios[
        (dataland_polls_2024_scenarios['scenario'] == scenario) & 
        (dataland_polls_2024_scenarios['geography'].isin([region, province]))
        ].drop('scenario', axis=1)
    # run model
    priors_list, final_daily_averages = run_forecast_pipeline(df_structural=df_str_province, 
                            df_structural_scenario = df_structural_scenario, 
                            df_polls = df_polls_province, 
                            df_poll_scenario = df_poll_scenario
                            )
    struct_dict_B[province] = priors_list
    final_daily_averages.insert(1, 'province', province)
    poll_results_B.append(final_daily_averages)

/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 9 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 9 seconds.
There were 4 divergences after tuning. Increase `target_accept` or reparameterize.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


...

/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 13 seconds.
The acceptance probability does not match the target. It is 0.8897688647110232, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8978732616796214, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 11 seconds.
The acceptance probability does not match the target. It is 0.9041758748635111, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 13 seconds.
The acceptance probability does not match the target. It is 0.9196956326621953, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9224424933717218, but should be close to 0.8. Try to increase the number of tuning steps.


In [102]:
# save model outcomes
with open('../model_outputs/struct_dict_B.json', 'w') as file:
    json.dump(struct_dict_B, file)

df_poll_results_B = pd.concat(poll_results_B)
df_poll_results_B.to_csv("../model_outputs/poll_results_B.csv", index=False)

In [103]:
scenario = 'C'
struct_dict_C = {} # stores province level results
poll_results_C = [] # stores province level results

for province in df_structural.province.unique():
    df_str_province = df_structural[df_structural['province'] == province]
    if province in ['Amperville', 'Circuiton', 'Electropolis', 'Voltagea']:
        region = 'Circuit Confederation'
    elif province in ['Cerebrica', 'Cortexia', 'Neuronia']:
        region = 'Synapse Territories'
    else: region = 'Metaflux Realm'

    df_polls_province = df_polls[df_polls['geography'].isin([region, province])]
    df_structural_scenario = dataland_economic_data_2024_scenarios[dataland_economic_data_2024_scenarios['scenario'] == scenario].drop('scenario', axis=1)
    df_structural_scenario['year'] = pd.DatetimeIndex(df_structural_scenario['date']).year
    df_poll_scenario = dataland_polls_2024_scenarios[
        (dataland_polls_2024_scenarios['scenario'] == scenario) & 
        (dataland_polls_2024_scenarios['geography'].isin([region, province]))
        ].drop('scenario', axis=1)
    # run model
    priors_list, final_daily_averages = run_forecast_pipeline(df_structural=df_str_province, 
                            df_structural_scenario = df_structural_scenario, 
                            df_polls = df_polls_province, 
                            df_poll_scenario = df_poll_scenario
                            )
    struct_dict_C[province] = priors_list
    final_daily_averages.insert(1, 'province', province)
    poll_results_C.append(final_daily_averages)

/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 9 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 10 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 13 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8865307799981897, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


...

/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 12 seconds.
The acceptance probability does not match the target. It is 0.8884776487305406, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9109908564137352, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 11 seconds.
The acceptance probability does not match the target. It is 0.9041758748635111, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 14 seconds.
The acceptance probability does not match the target. It is 0.9070851863646869, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9268671653604442, but should be close to 0.8. Try to increase the number of tuning steps.


In [104]:
# save model outcomes
with open('../model_outputs/struct_dict_C.json', 'w') as file:
    json.dump(struct_dict_C, file)

df_poll_results_C = pd.concat(poll_results_C)
df_poll_results_C.to_csv("../model_outputs/poll_results_C.csv", index=False)

In [105]:
scenario = 'D'
struct_dict_D = {} # stores province level results
poll_results_D = [] # stores province level results

for province in df_structural.province.unique():
    df_str_province = df_structural[df_structural['province'] == province]
    if province in ['Amperville', 'Circuiton', 'Electropolis', 'Voltagea']:
        region = 'Circuit Confederation'
    elif province in ['Cerebrica', 'Cortexia', 'Neuronia']:
        region = 'Synapse Territories'
    else: region = 'Metaflux Realm'

    df_polls_province = df_polls[df_polls['geography'].isin([region, province])]
    df_structural_scenario = dataland_economic_data_2024_scenarios[dataland_economic_data_2024_scenarios['scenario'] == scenario].drop('scenario', axis=1)
    df_structural_scenario['year'] = pd.DatetimeIndex(df_structural_scenario['date']).year
    df_poll_scenario = dataland_polls_2024_scenarios[
        (dataland_polls_2024_scenarios['scenario'] == scenario) & 
        (dataland_polls_2024_scenarios['geography'].isin([region, province]))
        ].drop('scenario', axis=1)
    # run model
    priors_list, final_daily_averages = run_forecast_pipeline(df_structural=df_str_province, 
                            df_structural_scenario = df_structural_scenario, 
                            df_polls = df_polls_province, 
                            df_poll_scenario = df_poll_scenario
                            )
    struct_dict_D[province] = priors_list
    final_daily_averages.insert(1, 'province', province)
    poll_results_D.append(final_daily_averages)

/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 9 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 9 seconds.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


...

/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 13 seconds.
The acceptance probability does not match the target. It is 0.9219591986235111, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 11 seconds.
The acceptance probability does not match the target. It is 0.9041758748635111, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 13 seconds.
The acceptance probability does not match the target. It is 0.9321531648829098, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9123739308145363, but should be close to 0.8. Try to increase the number of tuning steps.


In [106]:
# save model outcomes
with open('../model_outputs/struct_dict_D.json', 'w') as file:
    json.dump(struct_dict_D, file)

df_poll_results_D = pd.concat(poll_results_D)
df_poll_results_D.to_csv("../model_outputs/poll_results_D.csv", index=False)

In [107]:
scenario = 'E'
struct_dict_E = {} # stores province level results
poll_results_E = [] # stores province level results

for province in df_structural.province.unique():
    df_str_province = df_structural[df_structural['province'] == province]
    if province in ['Amperville', 'Circuiton', 'Electropolis', 'Voltagea']:
        region = 'Circuit Confederation'
    elif province in ['Cerebrica', 'Cortexia', 'Neuronia']:
        region = 'Synapse Territories'
    else: region = 'Metaflux Realm'

    df_polls_province = df_polls[df_polls['geography'].isin([region, province])]
    df_structural_scenario = dataland_economic_data_2024_scenarios[dataland_economic_data_2024_scenarios['scenario'] == scenario].drop('scenario', axis=1)
    df_structural_scenario['year'] = pd.DatetimeIndex(df_structural_scenario['date']).year
    df_poll_scenario = dataland_polls_2024_scenarios[
        (dataland_polls_2024_scenarios['scenario'] == scenario) & 
        (dataland_polls_2024_scenarios['geography'].isin([region, province]))
        ].drop('scenario', axis=1)
    # run model
    priors_list, final_daily_averages = run_forecast_pipeline(df_structural=df_str_province, 
                            df_structural_scenario = df_structural_scenario, 
                            df_polls = df_polls_province, 
                            df_poll_scenario = df_poll_scenario
                            )
    struct_dict_E[province] = priors_list
    final_daily_averages.insert(1, 'province', province)
    poll_results_E.append(final_daily_averages)

/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 8 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 10 seconds.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8908411357070561, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


...

/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 10 seconds.
The acceptance probability does not match the target. It is 0.9003462933096728, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.8925646793773774, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 12 seconds.
The acceptance probability does not match the target. It is 0.9041758748635111, but should be close to 0.8. Try to increase the number of tuning steps.


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [b, b0, theta]


/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/kbeebe/Desktop/economist_forecasting/election_forecaster/.venv/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 100 tune and 1_000 draw iterations (200 + 2_000 draws total) took 13 seconds.
The acceptance probability does not match the target. It is 0.9057724835660743, but should be close to 0.8. Try to increase the number of tuning steps.
The acceptance probability does not match the target. It is 0.9292290295220854, but should be close to 0.8. Try to increase the number of tuning steps.


In [108]:
# save model outcomes
with open('../model_outputs/struct_dict_E.json', 'w') as file:
    json.dump(struct_dict_E, file)

df_poll_results_E = pd.concat(poll_results_E)
df_poll_results_E.to_csv("../model_outputs/poll_results_E.csv", index=False)